<a href="https://colab.research.google.com/github/davidisinta/AI/blob/main/law_llms_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch z3-solver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install -q huggingface_hub

# Paste your token when prompted in the next cell
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
prompt = """
You are an expert in both legal reasoning and formal methods. Your task is to convert legal rules into executable SMT-LIB v2 logic that can be verified using the Z3 SMT solver.

Use the following guidelines:
- Declare all variables using `(declare-const <name> <type>)` with proper naming.
- Use `(define-fun <name> () Bool <body>)` to define boolean predicates.
- Use `and`, `or`, `not`, `=>`, and `=` for logical expressions.
- Use `Int` for numbers and `Bool` for true/false.
- Do not include comments or extra text—only valid SMT-LIB output.

-------------------
📘 EXAMPLE 1: Legal Rule → “A person is eligible to vote if they are over 18 and are a citizen.”

SMT Output:
(declare-const age Int)
(declare-const is_citizen Bool)

(define-fun is_eligible_to_vote () Bool
  (and (>= age 18) is_citizen))

-------------------
📘 EXAMPLE 2: Legal Rule → “An employee is eligible for benefits if they work more than 30 hours per week and have been employed for more than 180 days.”

SMT Output:
(declare-const hours_per_week Int)
(declare-const days_employed Int)

(define-fun is_eligible_for_benefits () Bool
  (and (> hours_per_week 30) (> days_employed 180)))

-------------------
📘 TASK: Convert the following U.S. legal rule into SMT-LIB v2:

26 USC §152(a)-(d)

(a) General Definition. – For purposes of this subtitle, the term “dependent” means—
  (1) a qualifying child, or
  (2) a qualifying relative.

(b) Qualifying Child. – The term “qualifying child” means, with respect to any taxpayer for any taxable year, an individual—
  (1) who bears a relationship to the taxpayer,
  (2) who has the same principal place of abode as the taxpayer for more than one-half of such taxable year,
  (3) who meets certain age requirements,
  (4) who has not provided over one-half of such individual’s own support for the calendar year,
  (5) who is not filing a joint return for the year.

(c) Qualifying Relative. – The term “qualifying relative” means, with respect to any taxpayer for any taxable year, an individual—
  (1) who bears a relationship to the taxpayer,
  (2) whose gross income for the year is less than the exemption amount,
  (3) who receives more than one-half of their support from the taxpayer,
  (4) who is not a qualifying child of the taxpayer or of any other taxpayer.

(d) Special Rule for Support. – In the case of multiple contributors, if no one individual contributes more than half, but a group does, and one person is designated, that person may claim the dependent.

-------------------
🎯 Now output the SMT-LIB v2 code below. Do not include explanations or extra text.
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=600,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
)

smt_code = tokenizer.decode(output[0], skip_special_tokens=True)
print(smt_code)



You are an expert in both legal reasoning and formal methods. Your task is to convert legal rules into executable SMT-LIB v2 logic that can be verified using the Z3 SMT solver.

Use the following guidelines:
- Declare all variables using `(declare-const <name> <type>)` with proper naming.
- Use `(define-fun <name> () Bool <body>)` to define boolean predicates.
- Use `and`, `or`, `not`, `=>`, and `=` for logical expressions.
- Use `Int` for numbers and `Bool` for true/false.
- Do not include comments or extra text—only valid SMT-LIB output.

-------------------
📘 EXAMPLE 1: Legal Rule → “A person is eligible to vote if they are over 18 and are a citizen.”

SMT Output:
(declare-const age Int)
(declare-const is_citizen Bool)

(define-fun is_eligible_to_vote () Bool
  (and (>= age 18) is_citizen))

-------------------
📘 EXAMPLE 2: Legal Rule → “An employee is eligible for benefits if they work more than 30 hours per week and have been employed for more than 180 days.”

SMT Output:
(declar

In [ ]:
def extract_smt_code_after_marker(full_text):
    marker = "Now output the SMT-LIB v2 code below. Do not include explanations or extra text."

    # Find the index of the marker
    start_idx = full_text.find(marker)
    if start_idx == -1:
        raise ValueError("Marker not found in text.")

    # Get everything after the marker
    smt_start = full_text[start_idx + len(marker):].strip()

    return smt_start


In [ ]:
raw_output = smt_code

# Extract the SMT only section
smt_code_cleaned = extract_smt_code_after_marker(raw_output)

# Write to file
with open("cleaned_generated_smt1.smt2", "w") as f:
    f.write(smt_code_cleaned)

# view the cleaned output
print(smt_code_cleaned)


(declare-const age Int)
(declare-const gross_income Int)
(declare-const support Int)
(declare-const days_employed Int)
(declare-const hours_per_week Int)
(declare-const relationship Bool)
(declare-const same_place_of_abode Bool)
(declare-const age_requirement Bool)
(declare-const support_requirement Bool)
(declare-const joint_return Bool)

(define-fun is_qualifying_child () Bool
  (and (< age 19) (or (= age 19) (= age 20)) (same_place_of_abode) (not (= relationship false)) (not (= support true)) (not (= joint_return true))))

(define-fun is_qualifying_relative () Bool
  (and (< gross_income 0) (not (= relationship false)) (not (= support true)) (not (= joint_return true)) (not (= age_requirement false)) (not (= support_requirement false))))

(define-fun is_dependent () Bool
  (or (is_qualifying_child) (is_qualifying_relative)))

(define-fun is_eligible_for_tax_credits () Bool
  (and (is_dependent) (not (= days_employed 0)) (not (= hours_per_week 0))))

(define-fun is_eligible_for_tax_e

Testing if generated smt is z3 compliant

In [ ]:
import subprocess

def run_z3_on_file(file_path):
    result = subprocess.run(["z3", file_path], capture_output=True, text=True)
    print("Z3 Output:")
    print(result.stdout)
    if result.stderr:
        print("Z3 Errors:")
        print(result.stderr)

run_z3_on_file("cleaned_generated_smt1.smt2")


Z3 Output:
(error "line 13 column 65: invalid function application, arguments missing")
(error "line 19 column 26: invalid function application, arguments missing")
(error "line 22 column 20: invalid function application, arguments missing")
(error "line 25 column 20: invalid function application, arguments missing")



I perform minor manual edits to the smt

In [ ]:
run_z3_on_file("smt_cleaned_part1.1")

Z3 Output:
sat
(
  (define-fun relationship () Bool
    false)
  (define-fun same_place_of_abode () Bool
    false)
  (define-fun gross_income () Int
    0)
  (define-fun age () Int
    0)
  (define-fun hours_per_week () Int
    0)
  (define-fun exemption_amount () Int
    0)
  (define-fun joint_return () Bool
    false)
  (define-fun support_requirement () Bool
    false)
  (define-fun provided_half_own_support () Bool
    false)
  (define-fun days_employed () Int
    0)
  (define-fun age_requirement () Bool
    false)
  (define-fun is_qualifying_relative () Bool
    (and (< gross_income exemption_amount)
     relationship
     support_requirement
     age_requirement
     (not joint_return)))
  (define-fun is_eligible_for_tax_exemptions () Bool
    (let ((a!1 (or (and (< age 19)
                    (or (= age 19) (= age 20))
                    same_place_of_abode
                    relationship
                    (not provided_half_own_support)
                    (not joint_retur

Part 2

In [ ]:
scenario_1 = """
John is 15 years old. He is the taxpayer’s biological son. He lived with the taxpayer for 250 days this year.
He did not provide more than half of his own support and did not file a joint return.
He is not a qualifying child of anyone else. His gross income is $0.
"""


In [ ]:
scenario_2 = """
Alice is 55 years old. She is a roommate of the taxpayer, not a relative.
She lived with the taxpayer for 300 days, filed a joint return with her spouse,
and provided most of her own support. Her gross income is $18,000.
"""


In [ ]:
scenario_3 = """
Marcus is a 19-year-old nephew of the taxpayer. He lived with the taxpayer for 185 days.
He earned $4,500, slightly below the exemption amount ($4,700).
He contributed about 48% to his own support.
He did not file a joint return. He is not a qualifying child of any other taxpayer.
"""


In [ ]:
!z3 scenario_1.smt2


sat
(
  (define-fun age_requirement () Bool
    true)
  (define-fun gross_income () Int
    0)
  (define-fun same_place_of_abode () Bool
    true)
  (define-fun hours_per_week () Int
    0)
  (define-fun joint_return () Bool
    false)
  (define-fun days_employed () Int
    0)
  (define-fun age () Int
    15)
  (define-fun relationship () Bool
    true)
  (define-fun support_requirement () Bool
    true)
  (define-fun lives_with_taxpayer_days () Int
    250)
  (define-fun is_qualifying_child_of_another () Bool
    false)
  (define-fun exemption_amount () Int
    4700)
  (define-fun taxpayer_provided_support () Bool
    true)
  (define-fun provided_half_own_support () Bool
    false)
  (define-fun is_qualifying_relative () Bool
    (and (< gross_income exemption_amount)
     relationship
     support_requirement
     age_requirement
     (not joint_return)))
  (define-fun is_eligible_for_tax_exemptions () Bool
    (let ((a!1 (or (and (< age 19)
                    (or (= age 19) (= age 

In [ ]:
!z3 scenario_2.smt2


unsat
(error "line 69 column 10: model is not available")


In [ ]:
!z3 scenario_3.smt2

sat
(
  (define-fun age_requirement () Bool
    true)
  (define-fun age () Int
    19)
  (define-fun relationship () Bool
    true)
  (define-fun gross_income () Int
    4500)
  (define-fun support_requirement () Bool
    true)
  (define-fun lives_with_taxpayer_days () Int
    185)
  (define-fun same_place_of_abode () Bool
    true)
  (define-fun hours_per_week () Int
    0)
  (define-fun exemption_amount () Int
    4700)
  (define-fun days_employed () Int
    0)
  (define-fun provided_half_own_support () Bool
    false)
  (define-fun taxpayer_provided_support () Bool
    true)
  (define-fun joint_return () Bool
    false)
  (define-fun is_qualifying_child_of_another () Bool
    false)
  (define-fun is_qualifying_relative () Bool
    (and (< gross_income exemption_amount)
     relationship
     support_requirement
     age_requirement
     (not joint_return)))
  (define-fun is_eligible_for_tax_exemptions () Bool
    (let ((a!1 (or (and (< age 19)
                    (or (= age 19) (= a